# 泰坦尼克号生存率预测
<font size=3>
1.项目基本信息

项目的背景是大家都熟知的发生在1912年的泰坦尼克号沉船灾难，这次灾难导致2224名船员和乘客中有1502人遇难。而哪些人幸存那些人丧生并非完全随机。比如说你碰巧搭乘了这艘游轮，而你碰巧又是一名人见人爱，花见花开的一等舱小公主，那活下来的概率就很大了，但是如果不巧你只是一名三等舱的抠脚大汉，那只有自求多福了。也就是说在这生死攸关的情况下，生存与否与性别，年龄，阶层等因素是有关系的，如果把这些因素作为特征，生存的结果作为预测目标，就可以建立一个典型的二分类机器学习模型。在这个项目中提供了部分的乘客名单，包括各种维度的特征以及是否幸存的标签，存在train.csv文件中，这是我们训练需要的数据；另一个test.csv文件是我们需要预测的乘客名单，只有相应的特征。我们要做的工作就是通过对训练数据的特征与生存关系进行探索，构建合适的机器学习的模型，再用这个模型预测测试文件中乘客的幸存情况，并将结果保存提交给kaggle。

<font size=3>
目录
    
1. 提出问题（Business Understanding ）
    
2. 理解数据（Data Understanding）
    
 * 采集数据
 * 导入数据
 * 查看数据集信息
3. 数据清洗（Data Preparation ）
    
 * 数据预处理
 * 特征工程（Feature Engineering）
    
4. 构建模型（Modeling） 
    
5. 模型评估（Evaluation） 
    
6. 方案实施 （Deployment）
    
 * 提交结果到Kaggle
 * 报告撰写

# 1.提出问题

什么样的人在泰坦尼克号中更容易存活？

# 2.理解数据

## 2.1 采集数据

从Kaggle泰坦尼克号项目页面下载数据：https://www.kaggle.com/c/titanic

## 2.2 导入数据

In [ ]:
# 忽略警告提示
import warnings
warnings.filterwarnings('ignore')

#导入处理数据包
import numpy as np
import pandas as pd

In [ ]:
#导入数据
#训练数据集
train = pd.read_csv("train.csv")
#测试数据集
test  = pd.read_csv("test.csv")
#这里要记住训练数据集有891条数据，方便后面从中拆分出测试数据集用于提交Kaggle结果
print ('训练数据集:',train.shape,'测试数据集:',test.shape)

In [ ]:
rowNum_train=train.shape[0]
rowNum_test=test.shape[0]
print('kaggle训练数据集有多少行数据：',rowNum_train,
     ',kaggle测试数据集有多少行数据：',rowNum_test,)

In [ ]:
#合并数据集，方便同时对两个数据集进行清洗
full = train.append( test , ignore_index = True )

print ('合并后的数据集:',full.shape)

## 2.3 查看数据集信息

In [ ]:
#查看数据
full.head()

In [ ]:
'''
describe只能查看数据类型的描述统计信息，对于其他类型的数据不显示，比如字符串类型姓名（name），客舱号（Cabin）
这很好理解，因为描述统计指标是计算数值，所以需要该列的数据类型是数据
'''
#获取数据类型列的描述统计信息
full.describe()

In [ ]:
# 查看每一列的数据类型，和数据总数
full.info()
'''
我们发现数据总共有1309行。
其中数据类型列：年龄（Age）、船舱号（Cabin）里面有缺失数据：
1）年龄（Age）里面数据总数是1046条，缺失了1309-1046=263，缺失率263/1309=20%
2）船票价格（Fare）里面数据总数是1308条，缺失了1条数据

字符串列：
1）登船港口（Embarked）里面数据总数是1307，只缺失了2条数据，缺失比较少
2）船舱号（Cabin）里面数据总数是295，缺失了1309-295=1014，缺失率=1014/1309=77.5%，缺失比较大
这为我们下一步数据清洗指明了方向，只有知道哪些数据缺失数据，我们才能有针对性的处理。
'''

# 3.数据清洗（Data Preparation ）

## 3.1 数据预处理

###  缺失值处理

在前面，理解数据阶段，我们发现数据总共有1309行。
其中数据类型列：年龄（Age）、船舱号（Cabin）里面有缺失数据。
字符串列：登船港口（Embarked）、船舱号（Cabin）里面有缺失数据。

这为我们下一步数据清洗指明了方向，只有知道哪些数据缺失数据，我们才能有针对性的处理。

很多机器学习算法为了训练模型，要求所传入的特征中不能有空值。


1. 如果是数值类型，用平均值取代
2. 如果是分类数据，用最常见的类别取代
3. 使用模型预测缺失值，例如：K-NN

In [ ]:
'''
我们发现数据总共有1309行。
其中数据类型列：年龄（Age）、船舱号（Cabin）里面有缺失数据：
1）年龄（Age）里面数据总数是1046条，缺失了1309-1046=263，缺失率263/1309=20%
2）船票价格（Fare）里面数据总数是1308条，缺失了1条数据

对于数据类型，处理缺失值最简单的方法就是用平均数来填充缺失值
'''
print('处理前：')
full.info()
#年龄(Age)
full['Age']=full['Age'].fillna( full['Age'].mean() )
#船票价格(Fare)
full['Fare'] = full['Fare'].fillna( full['Fare'].mean() )
print('处理后：')
full.info()

In [ ]:
#检查数据处理是否正常
full.head()

In [ ]:
'''
总数据是1309
字符串列：
1）登船港口（Embarked）里面数据总数是1307，只缺失了2条数据，缺失比较少
2）船舱号（Cabin）里面数据总数是295，缺失了1309-295=1014，缺失率=1014/1309=77.5%，缺失比较大
'''
#登船港口（Embarked）：查看里面数据长啥样
'''
出发地点：S=英国南安普顿Southampton
途径地点1：C=法国 瑟堡市Cherbourg
途径地点2：Q=爱尔兰 昆士敦Queenstown
'''
full['Embarked'].head()

In [ ]:
'''
分类变量Embarked，看下最常见的类别，用其填充
'''
full['Embarked'].value_counts()

In [ ]:
'''
从结果来看，S类别最常见。我们将缺失值填充为最频繁出现的值：
S=英国南安普顿Southampton
'''
full['Embarked'] = full['Embarked'].fillna( 'S' )

In [ ]:
#船舱号（Cabin）：查看里面数据长啥样
full['Cabin'].head()

In [ ]:
#缺失数据比较多，船舱号（Cabin）缺失值填充为U，表示未知（Uknow） 
full['Cabin'] = full['Cabin'].fillna( 'U' )

In [ ]:
#检查数据处理是否正常
full.head()

In [ ]:
#查看最终缺失值处理情况，记住生成情况（Survived）这里一列是我们的标签，用来做机器学习预测的，不需要处理这一列
full.info()

## 3.2 特征提取

### 3.2.1数据分类

查看数据类型，分为3种数据类型。并对类别数据处理：用数值代替类别，并进行One-hot编码

In [ ]:
'''
1.数值类型：
乘客编号（PassengerId），年龄（Age），船票价格（Fare），同代直系亲属人数（SibSp），不同代直系亲属人数（Parch）
2.时间序列：无
3.分类数据：
1）有直接类别的
乘客性别（Sex）：男性male，女性female
登船港口（Embarked）：出发地点S=英国南安普顿Southampton，途径地点1：C=法国 瑟堡市Cherbourg，出发地点2：Q=爱尔兰 昆士敦Queenstown
客舱等级（Pclass）：1=1等舱，2=2等舱，3=3等舱
2）字符串类型：可能从这里面提取出特征来，也归到分类数据中
乘客姓名（Name）
客舱号（Cabin）
船票编号（Ticket）
'''
full.info()

### 3.2.1 分类数据：有直接类别的

1. 乘客性别（Sex）：
男性male，女性female
2. 登船港口（Embarked）：出发地点S=英国南安普顿Southampton，途径地点1：C=法国 瑟堡市Cherbourg，出发地点2：Q=爱尔兰 昆士敦Queenstown
3. 客舱等级（Pclass）：1=1等舱，2=2等舱，3=3等舱

####  性别

In [ ]:
#查看性别数据这一列
full['Sex'].head()

In [ ]:
'''
将性别的值映射为数值
男（male）对应数值1，女（female）对应数值0
'''
sex_mapDict={'male':1,
            'female':0}
#map函数：对Series每个数据应用自定义的函数计算
full['Sex']=full['Sex'].map(sex_mapDict)
full.head()

#### 登船港口(Embarked)

In [ ]:
'''
登船港口(Embarked)的值是：
出发地点：S=英国南安普顿Southampton
途径地点1：C=法国 瑟堡市Cherbourg
途径地点2：Q=爱尔兰 昆士敦Queenstown
'''
#查看该类数据内容
full['Embarked'].head()

In [ ]:
#存放提取后的特征
embarkedDf = pd.DataFrame()

'''
使用get_dummies进行one-hot编码，产生虚拟变量（dummy variables），列名前缀是Embarked
'''
embarkedDf = pd.get_dummies( full['Embarked'] , prefix='Embarked' )
embarkedDf.head()

In [ ]:
#添加one-hot编码产生的虚拟变量（dummy variables）到泰坦尼克号数据集full
full = pd.concat([full,embarkedDf],axis=1)

'''
因为已经使用登船港口(Embarked)进行了one-hot编码产生了它的虚拟变量（dummy variables）
所以这里把登船港口(Embarked)删掉
'''
full.drop('Embarked',axis=1,inplace=True)
full.head()

In [ ]:
'''
上面drop删除某一列代码解释：
因为drop(name,axis=1)里面指定了name是哪一列，比如指定的是A这一列，axis=1表示按行操作。
那么结合起来就是把A列里面每一行删除，最终结果是删除了A这一列.
简单来说，使用drop删除某几列的方法记住这个语法就可以了：drop([列名1,列名2],axis=1)
'''

#### 客舱等级（Pclass）

In [ ]:
'''
客舱等级(Pclass):
1=1等舱，2=2等舱，3=3等舱
'''
#存放提取后的特征
pclassDf = pd.DataFrame()

#使用get_dummies进行one-hot编码，列名前缀是Pclass
pclassDf = pd.get_dummies( full['Pclass'] , prefix='Pclass' )
pclassDf.head()

In [ ]:
#添加one-hot编码产生的虚拟变量（dummy variables）到泰坦尼克号数据集full
full = pd.concat([full,pclassDf],axis=1)

#删掉客舱等级（Pclass）这一列
full.drop('Pclass',axis=1,inplace=True)
full.head()

### 3.2.1 分类数据：字符串类型

字符串类型：可能从这里面提取出特征来，也归到分类数据中，这里数据有：

1. 乘客姓名（Name）
2. 客舱号（Cabin）
3. 船票编号（Ticket）

### 从姓名中提取头衔

In [ ]:
'''
查看姓名这一列长啥样
注意到在乘客名字（Name）中，有一个非常显著的特点：
乘客头衔每个名字当中都包含了具体的称谓或者说是头衔，将这部分信息提取出来后可以作为非常有用一个新变量，可以帮助我们进行预测。
例如：
Braund, Mr. Owen Harris
Heikkinen, Miss. Laina
Oliva y Ocana, Dona. Fermina
Peter, Master. Michael J
'''
full[ 'Name' ].head()

In [ ]:
#练习从字符串中提取头衔，例如Mr
#split用于字符串分割，返回一个列表
#我们看到姓名中'Braund, Mr. Owen Harris'，逗号前面的是“名”，逗号后面是‘头衔. 姓’
name1='Braund, Mr. Owen Harris'
'''
split用于字符串按分隔符分割，返回一个列表。这里按逗号分隔字符串
也就是字符串'Braund, Mr. Owen Harris'被按分隔符,'拆分成两部分[Braund,Mr. Owen Harris]
你可以把返回的列表打印出来瞧瞧，这里获取到列表中元素序号为1的元素，也就是获取到头衔所在的那部分，即Mr. Owen Harris这部分
'''
#Mr. Owen Harris
str1=name1.split( ',' )[1] 
'''
继续对字符串Mr. Owen Harris按分隔符'.'拆分，得到这样一个列表[Mr, Owen Harris]
这里获取到列表中元素序号为0的元素，也就是获取到头衔所在的那部分Mr
'''
#Mr.
str2=str1.split( '.' )[0]
#strip() 方法用于移除字符串头尾指定的字符（默认为空格）
str3=str2.strip()

In [ ]:
'''
定义函数：从姓名中获取头衔
'''
def getTitle(name):
    str1=name.split( ',' )[1] #Mr. Owen Harris
    str2=str1.split( '.' )[0]#Mr
    #strip() 方法用于移除字符串头尾指定的字符（默认为空格）
    str3=str2.strip()
    return str3

In [ ]:
#存放提取后的特征
titleDf = pd.DataFrame()
#map函数：对Series每个数据应用自定义的函数计算
titleDf['Title'] = full['Name'].map(getTitle)
titleDf.head()

In [ ]:
'''
定义以下几种头衔类别：
Officer政府官员
Royalty王室（皇室）
Mr已婚男士
Mrs已婚妇女
Miss年轻未婚女子
Master有技能的人/教师
'''
#姓名中头衔字符串与定义头衔类别的映射关系
title_mapDict = {
                    "Capt":       "Officer",
                    "Col":        "Officer",
                    "Major":      "Officer",
                    "Jonkheer":   "Royalty",
                    "Don":        "Royalty",
                    "Sir" :       "Royalty",
                    "Dr":         "Officer",
                    "Rev":        "Officer",
                    "the Countess":"Royalty",
                    "Dona":       "Royalty",
                    "Mme":        "Mrs",
                    "Mlle":       "Miss",
                    "Ms":         "Mrs",
                    "Mr" :        "Mr",
                    "Mrs" :       "Mrs",
                    "Miss" :      "Miss",
                    "Master" :    "Master",
                    "Lady" :      "Royalty"
                    }

#map函数：对Series每个数据应用自定义的函数计算
titleDf['Title'] = titleDf['Title'].map(title_mapDict)

#使用get_dummies进行one-hot编码
titleDf = pd.get_dummies(titleDf['Title'])
titleDf.head()

In [ ]:
#添加one-hot编码产生的虚拟变量（dummy variables）到泰坦尼克号数据集full
full = pd.concat([full,titleDf],axis=1)

#删掉姓名这一列
full.drop('Name',axis=1,inplace=True)
full.head()

### 从客舱号中提取客舱类别

In [ ]:
#补充知识：匿名函数
'''
python 使用 lambda 来创建匿名函数。
所谓匿名，意即不再使用 def 语句这样标准的形式定义一个函数，预防如下：
lambda 参数1，参数2：函数体或者表达式
'''
# 定义匿名函数：对两个数相加
sum = lambda a,b: a + b
 
# 调用sum函数
print ("相加后的值为 : ", sum(10,20))

In [ ]:
'''
客舱号的首字母是客舱的类别
'''
#查看客舱号的内容
full['Cabin'].head()

In [ ]:
#存放客舱号信息
cabinDf = pd.DataFrame()

'''
客场号的类别值是首字母，例如：
C85 类别映射为首字母C
'''
full[ 'Cabin' ] = full[ 'Cabin' ].map(lambda c : c[0] )

##使用get_dummies进行one-hot编码，列名前缀是Cabin
cabinDf = pd.get_dummies( full['Cabin'] , prefix = 'Cabin' )

cabinDf.head()

In [ ]:
#添加one-hot编码产生的虚拟变量（dummy variables）到泰坦尼克号数据集full
full = pd.concat([full,cabinDf],axis=1)

#删掉客舱号这一列
full.drop('Cabin',axis=1,inplace=True)
full.head()

### 建立家庭人数和家庭类别

In [ ]:
#存放家庭信息
familyDf = pd.DataFrame()

'''
家庭人数=同代直系亲属数（Parch）+不同代直系亲属数（SibSp）+乘客自己
（因为乘客自己也是家庭成员的一个，所以这里加1）
'''
familyDf[ 'FamilySize' ] = full[ 'Parch' ] + full[ 'SibSp' ] + 1

'''
家庭类别：
小家庭Family_Single：家庭人数=1
中等家庭Family_Small: 2<=家庭人数<=4
大家庭Family_Large: 家庭人数>=5
'''
#if 条件为真的时候返回if前面内容，否则返回0
familyDf[ 'Family_Single' ] = familyDf[ 'FamilySize' ].map( lambda s : 1 if s == 1 else 0 )
familyDf[ 'Family_Small' ]  = familyDf[ 'FamilySize' ].map( lambda s : 1 if 2 <= s <= 4 else 0 )
familyDf[ 'Family_Large' ]  = familyDf[ 'FamilySize' ].map( lambda s : 1 if 5 <= s else 0 )

familyDf.head()

In [ ]:
#添加one-hot编码产生的虚拟变量（dummy variables）到泰坦尼克号数据集full
full = pd.concat([full,familyDf],axis=1)
full.head()

In [ ]:
#到现在我们已经有了这么多个特征了
full.shape

## 3.3 特征选择

可以学习后面的课程后，再了解特征选择的方法。但是如果你已经具备了多种机器学习算法的知识，想提前学习，可以参考这些资料：
 
* [如何做特征工程?](http://www.csuldw.com/2015/10/24/2015-10-24%20feature%20engineering/)
* [如何使用sklearn进行特征工程？](http://www.cnblogs.com/jasonfreak/p/5448385.html)

* [泰坦尼克号如何进行特征选择?](https://ahmedbesbes.com/how-to-score-08134-in-titanic-kaggle-challenge.html)

相关系数法：计算各个特征的相关系数

In [ ]:
#相关性矩阵
corrDf = full.corr() 
corrDf

In [ ]:
'''
查看各个特征与生成情况（Survived）的相关系数，
ascending=False表示按降序排列
'''
corrDf['Survived'].sort_values(ascending =False)

根据各个特征与生成情况（Survived）的相关系数大小，我们选择了这几个特征作为模型的输入：

头衔（前面所在的数据集titleDf）、客舱等级（pclassDf）、家庭大小（familyDf）、船票价格（Fare）、船舱号（cabinDf）、登船港口（embarkedDf）、性别（Sex）

In [ ]:
#特征选择
full_X = pd.concat( [titleDf,#头衔
                     pclassDf,#客舱等级
                     familyDf,#家庭大小
                     full['Fare'],#船票价格
                     cabinDf,#船舱号
                     embarkedDf,#登船港口
                     full['Sex']#性别
                    ] , axis=1 )
full_X.head()

# 4.构建模型

用训练数据和某个机器学习算法得到机器学习模型，用测试数据评估模型

## 4.1 建立训练数据集和测试数据集

In [ ]:
'''
1）坦尼克号测试数据集因为是我们最后要提交给Kaggle的，里面没有生存情况的值，所以不能用于评估模型。
我们将Kaggle泰坦尼克号项目给我们的测试数据，叫做预测数据集（记为pred,也就是预测英文单词predict的缩写）。
也就是我们使用机器学习模型来对其生存情况就那些预测。
2）我们使用Kaggle泰坦尼克号项目给的训练数据集，做为我们的原始数据集（记为source），
从这个原始数据集中拆分出训练数据集（记为train：用于模型训练）和测试数据集（记为test：用于模型评估）。

'''
#原始数据集有891行
sourceRow=891

'''
sourceRow是我们在最开始合并数据前知道的，原始数据集有总共有891条数据
从特征集合full_X中提取原始数据集提取前891行数据时，我们要减去1，因为行号是从0开始的。
'''
#原始数据集：特征
source_X = full_X.loc[0:sourceRow-1,:]
#原始数据集：标签
source_y = full.loc[0:sourceRow-1,'Survived']   

#预测数据集：特征
pred_X = full_X.loc[sourceRow:,:]
'''
上面代码解释：
891行前面的数据是测试数据集，891行之后的数据是预测数据集。[sourceRow:,:]就是从891行开始到最后一行作为预测数据集
'''

In [ ]:
'''
确保这里原始数据集取的是前891行的数据，不然后面模型会有错误
'''
#原始数据集有多少行
print('原始数据集有多少行:',source_X.shape[0])
#预测数据集大小
print('原始数据集有多少行:',pred_X.shape[0])

In [ ]:
'''
从原始数据集（source）中拆分出训练数据集（用于模型训练train），测试数据集（用于模型评估test）
train_test_split是交叉验证中常用的函数，功能是从样本中随机的按比例选取train data和test data
train_data：所要划分的样本特征集
train_target：所要划分的样本结果
test_size：样本占比，如果是整数的话就是样本的数量
'''
from sklearn.model_selection import train_test_split 

#建立模型用的训练数据集和测试数据集
train_X, test_X, train_y, test_y = train_test_split(source_X ,
                                                    source_y,
                                                    train_size=.8)

#输出数据集大小
print ('原始数据集特征：',source_X.shape, 
       '训练数据集特征：',train_X.shape ,
      '测试数据集特征：',test_X.shape)

print ('原始数据集标签：',source_y.shape, 
       '训练数据集标签：',train_y.shape ,
      '测试数据集标签：',test_y.shape)

In [ ]:
#原始数据查看
source_y.head()

## 4.2 选择机器学习算法

选择一个机器学习算法，用于模型的训练。如果你是新手，建议从逻辑回归算法开始

In [ ]:
#第1步：导入算法
from sklearn.linear_model import LogisticRegression
#第2步：创建模型：逻辑回归（logisic regression）
model = LogisticRegression()

In [ ]:
#随机森林Random Forests Model
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model2 = RandomForestClassifier(n_estimators=100)

In [ ]:
#支持向量机Support Vector Machines
#from sklearn.svm import SVC, LinearSVC
#model3 = SVC()

In [ ]:
#Gradient Boosting Classifier
#from sklearn.ensemble import GradientBoostingClassifier
#model4 = GradientBoostingClassifier()

In [ ]:
#K-nearest neighbors
#from sklearn.neighbors import KNeighborsClassifier
#model5 = KNeighborsClassifier(n_neighbors = 3)

In [ ]:
# Gaussian Naive Bayes
#from sklearn.naive_bayes import GaussianNB
#model6 = GaussianNB()

## 4.3 训练模型

In [ ]:
#第3步：训练模型
model.fit( train_X , train_y )

## 5.评估模型

In [ ]:
# 分类问题，score得到的是模型的正确率
model.score(test_X , test_y )

# 6.方案实施（Deployment）

## 6.1 得到预测结果上传到Kaggle

使用预测数据集到底预测结果，并保存到csv文件中，上传到Kaggle中，就可以看到排名。

In [ ]:
#使用机器学习模型，对预测数据集中的生存情况进行预测
pred_Y = model.predict(pred_X)

'''
生成的预测值是浮点数（0.0,1,0）
但是Kaggle要求提交的结果是整型（0,1）
所以要对数据类型进行转换
'''
pred_Y=pred_Y.astype(int)
#乘客id
passenger_id = full.loc[sourceRow:,'PassengerId']
#数据框：乘客id，预测生存情况的值
predDf = pd.DataFrame( 
    { 'PassengerId': passenger_id , 
     'Survived': pred_Y } )
predDf.shape
predDf.head()
#保存结果
predDf.to_csv( 'titanic_pred.csv' , index = False )